**Parsing Notes**

Some entries seem to be missing context

In [17]:
from bs4 import BeautifulSoup as Soup
from IPython.display import display, HTML
import pandas as pd
import re
import pprint
import json

In [18]:
fn = 'procPublicationRequest Oct-Dec 2014 (Updated) - Sheet1-2.csv'
rows = pd.read_csv(fn, header=0)

In [19]:
def scrape(row):
    output = {}
    if row['AgencyName'] == "Citywide Administrative Services" and \
       row['TypeOfNoticeDescription'] == "Public Hearings":
        try:
            if not isinstance(row.AdditionalDescription, str):
                output = { 'error' : row.AdditionalDescription }
            else:    
                soup = Soup(row.AdditionalDescription)
                text = soup.get_text(strip=True)
                print('BEGIN: ' +'=-=-=-=-==-==-='*7)
                pprint.pprint(soup.prettify())
                print('---'*22)
                display(HTML(row.AdditionalDescription))
                print('---'*22)
                print('END: ' + '=-=-=-=-==-==-='*7)
                display(HTML('<br/><hr/><br/>'))
        except Exception:
            output = { 'error' : 'bad input', 'text': text }
    row['output'] = json.dumps(output)
    return row

In [20]:
fix = rows.apply(scrape,1)
# fix

BEGIN: =-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-=
('<p align=\'center"\'>\n'
 '</p>\n'
 '<p>\n'
 ' <strong>\n'
 '  NOTICE IS HEREBY GIVEN THAT A REAL PROPERTY ACQUISITIONS AND '
 'DISPOSITIONS PUBLIC HEARING\n'
 ' </strong>\n'
 '</p>')
------------------------------------------------------------------


------------------------------------------------------------------
END: =-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-=


BEGIN: =-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-=
('<p>\n'
 ' <strong>\n'
 '  NOTICE IS HEREBY GIVEN THAT A REAL PROPERTY ACQUISITIONS AND '
 'DISPOSITIONS PUBLIC HEARING\n'
 ' </strong>\n'
 ' , in accordance with Section 824 of the New York City Charter, will be '
 'held on Wednesday October 22, 2014 at 10:00 a.m., 22 Reade Street, 2\n'
 ' <sup>\n'
 '  nd\n'
 ' </sup>\n'
 ' floor conference room, Borough of Manhattan\n'
 '</p>\n'
 '<p>\n'
 ' <strong>\n'
 '  IN THE MATTER OF\n'
 ' </strong>\n'
 ' a lease for The City of New York, as Tenant, of approximately 10,826 '
 'rentable square feet of space on part of the second\xa0 floor in a '
 'building located at 1 Teleport Drive\xa0 (Block 2165, Lot 170), in the '
 'Borough of Staten Island, for the Traffic Enforcement Division of the '
 'Police Department to use as an office.\n'
 '</p>\n'
 '<p>\n'
 '</p>\n'
 '<p>\n'
 ' The proposed lease shall be for a period of twenty (20) yea

------------------------------------------------------------------
END: =-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-=


BEGIN: =-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-=
('<p align=\'center"\'>\n'
 ' <strong>\n'
 '  Notice of Public Hearing\n'
 ' </strong>\n'
 '</p>\n'
 '<p>\n'
 '</p>\n'
 '<p>\n'
 ' <strong>\n'
 '  NOTICE IS HEREBY GIVEN THAT A REAL PROPERTY ACQUISITIONS AND '
 'DISPOSITIONS PUBLIC HEARING\n'
 ' </strong>\n'
 '</p>')
------------------------------------------------------------------


...

------------------------------------------------------------------
END: =-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-=


BEGIN: =-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-=
('<p align=\'center"\'>\n'
 '</p>\n'
 '<p>\n'
 ' <strong>\n'
 '  NOTICE IS HEREBY GIVEN THAT A REAL PROPERTY ACQUISITIONS AND '
 'DISPOSITIONS PUBLIC HEARING\n'
 ' </strong>\n'
 '</p>')
------------------------------------------------------------------


------------------------------------------------------------------
END: =-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-=


BEGIN: =-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-=
('<p align=\'center"\'>\n'
 ' <strong>\n'
 '  Corrected Notice of Public Hearing\n'
 ' </strong>\n'
 '</p>\n'
 '<p>\n'
 '</p>\n'
 '<p>\n'
 ' <strong>\n'
 '  NOTICE IS HEREBY GIVEN THAT A REAL PROPERTY ACQUISITIONS AND '
 'DISPOSITIONS PUBLIC HEARING\n'
 ' </strong>\n'
 '</p>')
------------------------------------------------------------------


------------------------------------------------------------------
END: =-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-==-=-=-=-==-==-=
